## Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from ipaddress import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score,f1_score,auc,roc_curve,roc_auc_score, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding, Dropout, Flatten, SpatialDropout1D, LSTM,Input, concatenate, Reshape, InputLayer, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.regularizers import l1, l2, l1_l2
import keras
import re
import tldextract
import warnings
warnings.filterwarnings("ignore")
sns.set_style(style='white') 

/home/lab_services_student/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-10-13 11:13:23.315905: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-13 11:13:23.317148: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-13 11:13:23.320518: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-13 11:13:24.472412: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting

## Data Transforming

In [2]:

data=pd.read_csv('cleaned_data.csv')
df=data[['url','status']]
data=data.drop('url',axis=1)

In [3]:
data

,status,url_length,num_special_chars,num_digits,num_dots,num_slash,num_and,num_percent,domain_length,num_digits_in_domain,has_subdomain,is_HTTPS,depth,top_domain_encoded,domain_encoded
0,0,39,0,7,2,0,0,0,12,0,1,0,1,225,111707
1,0,36,0,6,2,0,0,0,12,0,1,0,1,225,111707
2,0,12,0,5,1,0,0,0,5,5,0,0,1,878,10
3,0,35,0,4,2,0,0,0,12,0,1,0,1,225,111707
4,0,20,0,5,2,0,0,0,8,0,1,0,1,225,291146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
798458,0,9,0,0,1,0,0,0,5,0,0,0,1,225,308751
798459,0,6,0,0,1,0,0,0,3,0,0,0,1,690,308750
798460,0,9,0,0,2,0,0,0,3,0,0,0,1,215,308771
798461,0,16,0,0,2,0,0,0,10,0,0,0,1,219,308776


In [4]:
a=data.top_domain_encoded.nunique()
data.top_domain_encoded.nunique()

1232

In [5]:
b=data.domain_encoded.nunique()
data.domain_encoded.nunique()

308783

In [6]:
# c=data.subdomain_encoded.nunique()
# data.subdomain_encoded.nunique()

In [7]:
d=a+b
d

310015

In [8]:
data.head()

,status,url_length,num_special_chars,num_digits,num_dots,num_slash,num_and,num_percent,domain_length,num_digits_in_domain,has_subdomain,is_HTTPS,depth,top_domain_encoded,domain_encoded
0,0,39,0,7,2,0,0,0,12,0,1,0,1,225,111707
1,0,36,0,6,2,0,0,0,12,0,1,0,1,225,111707
2,0,12,0,5,1,0,0,0,5,5,0,0,1,878,10
3,0,35,0,4,2,0,0,0,12,0,1,0,1,225,111707
4,0,20,0,5,2,0,0,0,8,0,1,0,1,225,291146


In [9]:
y=data['status']
vectorizer = TfidfVectorizer(analyzer='char')
X=vectorizer.fit_transform(df.url)

In [10]:
X.shape

(798463, 266)

In [11]:
X=X.toarray()
print(X.shape)
X=X.reshape(X.shape[0],14,19)

(798463, 266)


In [12]:
X_train, X_test, y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=54)
X_train.shape


(638770, 14, 19)

In [13]:
# X_train = pad_sequences(X_train, padding='post', truncating='post')
# X_test = pad_sequences(X_test, padding='post', truncating='post')

## CNN

In [14]:
tf.keras.backend.clear_session()
tf.random.set_seed(50)

In [15]:
cnn  = Sequential()
cnn.add(Conv1D(128,(3), input_shape=(14,19), activation='relu'))
cnn.add(Dropout(0.2))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(24,activation='relu'))
# cnn.add(Dense(1,activation ='sigmoid',kernel_regularizer=l1(0.01), bias_regularizer=l2(0.01)))
cnn.add(Dense(1,activation ='sigmoid'))


In [16]:
cnn.compile(loss ='binary_crossentropy',optimizer='adam', metrics=['accuracy', 'precision', 'recall'])

In [17]:
num_epochs =50
early_stop = EarlyStopping(monitor='val_loss',patience=3, verbose=1, restore_best_weights=True)
historyC=cnn.fit(X_train, y_train,epochs = num_epochs,validation_data=(X_test,y_test),callbacks =[early_stop],verbose =2)


Epoch 1/50
19962/19962 - 64s - 3ms/step - accuracy: 0.8986 - loss: 0.2475 - precision: 0.8853 - recall: 0.9311 - val_accuracy: 0.9173 - val_loss: 0.2074 - val_precision: 0.9007 - val_recall: 0.9499
Epoch 2/50
19962/19962 - 60s - 3ms/step - accuracy: 0.9146 - loss: 0.2118 - precision: 0.9021 - recall: 0.9426 - val_accuracy: 0.9222 - val_loss: 0.1964 - val_precision: 0.9099 - val_recall: 0.9483
Epoch 3/50
19962/19962 - 60s - 3ms/step - accuracy: 0.9187 - loss: 0.2017 - precision: 0.9066 - recall: 0.9455 - val_accuracy: 0.9245 - val_loss: 0.1891 - val_precision: 0.9199 - val_recall: 0.9405
Epoch 4/50
19962/19962 - 60s - 3ms/step - accuracy: 0.9215 - loss: 0.1952 - precision: 0.9100 - recall: 0.9468 - val_accuracy: 0.9266 - val_loss: 0.1835 - val_precision: 0.9151 - val_recall: 0.9509
Epoch 5/50
19962/19962 - 61s - 3ms/step - accuracy: 0.9233 - loss: 0.1915 - precision: 0.9123 - recall: 0.9477 - val_accuracy: 0.9279 - val_loss: 0.1814 - val_precision: 0.9199 - val_recall: 0.9475
Epoch 6/50

In [18]:
results = cnn.evaluate(X_test, y_test, verbose=2)
y_pred_prob = cnn.predict(X_test)  # Get the predicted probabilities
y_pred = (y_pred_prob > 0.5).astype(int)  
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)  
# Assuming you compiled with accuracy, precision, and recall
print(f"Restored Model Test Loss: {results[0]}")
print(f"Restored Model Test Accuracy: {results[1]}")
print(f"Restored Model Test Precision: {results[2]}")
print(f"Restored Model Test Recall: {results[3]}")
print(f"F1 Score: {f1}")
print(f"ROC-AUC Score: {roc_auc}")

4991/4991 - 9s - 2ms/step - accuracy: 0.9346 - loss: 0.1644 - precision: 0.9258 - recall: 0.9540
4991/4991 ━━━━━━━━━━━━━━━━━━━━ 9s 2ms/step
Restored Model Test Loss: 0.16442710161209106
Restored Model Test Accuracy: 0.9345807433128357
Restored Model Test Precision: 0.9258096218109131
Restored Model Test Recall: 0.9539924263954163
F1 Score: 0.9396897604793877
ROC-AUC Score: 0.9823674967982562
